## Student 1

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_1_EYE = pd.read_csv('data/STData/1/1_EYE.csv')

In [ ]:
df_1_EYE.head()

In [ ]:
df_1_EYE.shape

In [ ]:
df_1_EYE.columns

In [ ]:
df_1_EYE.info()

In [ ]:
df_1_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_1_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_1_EYE['QuestionKey'].unique()

In [ ]:
df_1_EYE['Timestamp'] = pd.to_datetime(df_1_EYE['Timestamp'])

In [ ]:
df_1_EYE.head(3)

In [ ]:
df_1_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_1_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_1_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_1_EYE.isnull().sum()

In [ ]:
df_1_EYE.dropna(inplace=True)

In [ ]:
df_1_EYE.head()

In [ ]:
df_1_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_1_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_1_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_1_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_1_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_1_EYE['ET_ValidityRight'].unique()

In [ ]:
df_1_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_1_EYE['ET_ValidityLeft'].value_counts().index, y=df_1_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_1_EYE['ET_ValidityRight'].value_counts().index, y=df_1_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_1_EYE['ET_ValidityLeft'] = df_1_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_1_EYE['ET_ValidityRight'] = df_1_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_1_EYE.head(3)

In [ ]:
df_1_EYE.describe()

In [ ]:
df_1_EYE[df_1_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_1_EYE[df_1_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_1_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_1_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilLeft'] == -1].shape[0] / df_1_EYE.shape[0]

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilRight'] == -1].shape[0] / df_1_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_1_EYE[df_1_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_1_EYE[df_1_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_1_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_1_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_1_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_1_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_1_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_1_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_1_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_1_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_1_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_1_EYE['Timestamp'], df_1_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_1_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_1_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_1_EYE[col].fillna(df_1_EYE[col].mean(), inplace=True)

In [ ]:
df_1_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_1_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_1_EYE = pd.read_csv('data/STData/1/1_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_1_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_1_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_1_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_1_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_1_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_1_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_1_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_1_EYE['Timestamp'] = pd.to_datetime(df_1_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_1_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_1_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_1_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_1_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_1_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_1_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_1_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_1_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_1_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_1_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_1_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_1_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_1_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_1_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_1_EYE['ET_ValidityLeft'].value_counts().index, y=df_1_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_1_EYE['ET_ValidityRight'].value_counts().index, y=df_1_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_1_EYE['ET_ValidityLeft'] = df_1_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_1_EYE['ET_ValidityRight'] = df_1_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_1_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_1_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_1_EYE[df_1_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_1_EYE[df_1_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_1_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_1_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilLeft'] == -1].shape[0] / df_1_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_1_EYE[df_1_EYE['ET_PupilRight'] == -1].shape[0] / df_1_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_1_EYE[df_1_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_1_EYE[df_1_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_1_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_1_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_1_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_1_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_1_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_1_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_1_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_1_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_1_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_1_EYE['Timestamp'], df_1_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_1_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_1_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_1_EYE[col].fillna(df_1_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_1_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_1_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_1_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 2

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_2_EYE = pd.read_csv('data/STData/2/2_EYE.csv')

In [ ]:
df_2_EYE.head()

In [ ]:
df_2_EYE.shape

In [ ]:
df_2_EYE.columns

In [ ]:
df_2_EYE.info()

In [ ]:
df_2_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_2_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_2_EYE['QuestionKey'].unique()

In [ ]:
df_2_EYE['Timestamp'] = pd.to_datetime(df_2_EYE['Timestamp'])

In [ ]:
df_2_EYE.head(3)

In [ ]:
df_2_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_2_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_2_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_2_EYE.isnull().sum()

In [ ]:
df_2_EYE.dropna(inplace=True)

In [ ]:
df_2_EYE.head()

In [ ]:
df_2_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_2_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_2_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_2_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_2_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_2_EYE['ET_ValidityRight'].unique()

In [ ]:
df_2_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_2_EYE['ET_ValidityLeft'].value_counts().index, y=df_2_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_2_EYE['ET_ValidityRight'].value_counts().index, y=df_2_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_2_EYE['ET_ValidityLeft'] = df_2_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_2_EYE['ET_ValidityRight'] = df_2_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_2_EYE.head(3)

In [ ]:
df_2_EYE.describe()

In [ ]:
df_2_EYE[df_2_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_2_EYE[df_2_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_2_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_2_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilLeft'] == -1].shape[0] / df_2_EYE.shape[0]

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilRight'] == -1].shape[0] / df_2_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_2_EYE[df_2_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_2_EYE[df_2_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_2_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_2_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_2_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_2_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_2_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_2_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_2_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_2_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_2_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_2_EYE['Timestamp'], df_2_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_2_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_2_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_2_EYE[col].fillna(df_2_EYE[col].mean(), inplace=True)

In [ ]:
df_2_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_2_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_2_EYE = pd.read_csv('data/STData/2/2_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_2_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_2_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_2_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_2_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_2_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_2_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_2_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_2_EYE['Timestamp'] = pd.to_datetime(df_2_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_2_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_2_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_2_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_2_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_2_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_2_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_2_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_2_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_2_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_2_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_2_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_2_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_2_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_2_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_2_EYE['ET_ValidityLeft'].value_counts().index, y=df_2_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_2_EYE['ET_ValidityRight'].value_counts().index, y=df_2_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_2_EYE['ET_ValidityLeft'] = df_2_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_2_EYE['ET_ValidityRight'] = df_2_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_2_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_2_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_2_EYE[df_2_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_2_EYE[df_2_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_2_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_2_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilLeft'] == -1].shape[0] / df_2_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_2_EYE[df_2_EYE['ET_PupilRight'] == -1].shape[0] / df_2_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_2_EYE[df_2_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_2_EYE[df_2_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_2_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_2_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_2_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_2_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_2_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_2_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_2_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_2_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_2_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_2_EYE['Timestamp'], df_2_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_2_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_2_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_2_EYE[col].fillna(df_2_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_2_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_2_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_2_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 3

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_3_EYE = pd.read_csv('data/STData/3/3_EYE.csv')

In [ ]:
df_3_EYE.head()

In [ ]:
df_3_EYE.shape

In [ ]:
df_3_EYE.columns

In [ ]:
df_3_EYE.info()

In [ ]:
df_3_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_3_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_3_EYE['QuestionKey'].unique()

In [ ]:
df_3_EYE['Timestamp'] = pd.to_datetime(df_3_EYE['Timestamp'])

In [ ]:
df_3_EYE.head(3)

In [ ]:
df_3_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_3_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_3_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_3_EYE.isnull().sum()

In [ ]:
df_3_EYE.dropna(inplace=True)

In [ ]:
df_3_EYE.head()

In [ ]:
df_3_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_3_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_3_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_3_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_3_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_3_EYE['ET_ValidityRight'].unique()

In [ ]:
df_3_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_3_EYE['ET_ValidityLeft'].value_counts().index, y=df_3_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_3_EYE['ET_ValidityRight'].value_counts().index, y=df_3_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_3_EYE['ET_ValidityLeft'] = df_3_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_3_EYE['ET_ValidityRight'] = df_3_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_3_EYE.head(3)

In [ ]:
df_3_EYE.describe()

In [ ]:
df_3_EYE[df_3_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_3_EYE[df_3_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_3_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_3_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilLeft'] == -1].shape[0] / df_3_EYE.shape[0]

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilRight'] == -1].shape[0] / df_3_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_3_EYE[df_3_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_3_EYE[df_3_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_3_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_3_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_3_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_3_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_3_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_3_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_3_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_3_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_3_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_3_EYE['Timestamp'], df_3_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_3_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_3_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_3_EYE[col].fillna(df_3_EYE[col].mean(), inplace=True)

In [ ]:
df_3_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_3_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_3_EYE = pd.read_csv('data/STData/3/3_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_3_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_3_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_3_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_3_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_3_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_3_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_3_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_3_EYE['Timestamp'] = pd.to_datetime(df_3_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_3_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_3_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_3_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_3_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_3_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_3_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_3_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_3_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_3_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_3_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_3_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_3_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_3_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_3_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_3_EYE['ET_ValidityLeft'].value_counts().index, y=df_3_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_3_EYE['ET_ValidityRight'].value_counts().index, y=df_3_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_3_EYE['ET_ValidityLeft'] = df_3_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_3_EYE['ET_ValidityRight'] = df_3_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_3_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_3_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_3_EYE[df_3_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_3_EYE[df_3_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_3_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_3_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilLeft'] == -1].shape[0] / df_3_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_3_EYE[df_3_EYE['ET_PupilRight'] == -1].shape[0] / df_3_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_3_EYE[df_3_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_3_EYE[df_3_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_3_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_3_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_3_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_3_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_3_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_3_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_3_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_3_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_3_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_3_EYE['Timestamp'], df_3_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_3_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_3_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_3_EYE[col].fillna(df_3_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_3_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_3_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_3_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 4

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_4_EYE = pd.read_csv('data/STData/4/4_EYE.csv')

In [ ]:
df_4_EYE.head()

In [ ]:
df_4_EYE.shape

In [ ]:
df_4_EYE.columns

In [ ]:
df_4_EYE.info()

In [ ]:
df_4_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_4_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_4_EYE['QuestionKey'].unique()

In [ ]:
df_4_EYE['Timestamp'] = pd.to_datetime(df_4_EYE['Timestamp'])

In [ ]:
df_4_EYE.head(3)

In [ ]:
df_4_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_4_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_4_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_4_EYE.isnull().sum()

In [ ]:
df_4_EYE.dropna(inplace=True)

In [ ]:
df_4_EYE.head()

In [ ]:
df_4_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_4_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_4_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_4_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_4_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_4_EYE['ET_ValidityRight'].unique()

In [ ]:
df_4_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_4_EYE['ET_ValidityLeft'].value_counts().index, y=df_4_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_4_EYE['ET_ValidityRight'].value_counts().index, y=df_4_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_4_EYE['ET_ValidityLeft'] = df_4_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_4_EYE['ET_ValidityRight'] = df_4_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_4_EYE.head(3)

In [ ]:
df_4_EYE.describe()

In [ ]:
df_4_EYE[df_4_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_4_EYE[df_4_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_4_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_4_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilLeft'] == -1].shape[0] / df_4_EYE.shape[0]

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilRight'] == -1].shape[0] / df_4_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_4_EYE[df_4_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_4_EYE[df_4_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_4_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_4_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_4_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_4_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_4_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_4_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_4_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_4_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_4_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_4_EYE['Timestamp'], df_4_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_4_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_4_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_4_EYE[col].fillna(df_4_EYE[col].mean(), inplace=True)

In [ ]:
df_4_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_4_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_4_EYE = pd.read_csv('data/STData/4/4_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_4_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_4_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_4_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_4_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_4_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_4_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_4_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_4_EYE['Timestamp'] = pd.to_datetime(df_4_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_4_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_4_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_4_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_4_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_4_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_4_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_4_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_4_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_4_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_4_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_4_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_4_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_4_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_4_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_4_EYE['ET_ValidityLeft'].value_counts().index, y=df_4_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_4_EYE['ET_ValidityRight'].value_counts().index, y=df_4_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_4_EYE['ET_ValidityLeft'] = df_4_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_4_EYE['ET_ValidityRight'] = df_4_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_4_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_4_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_4_EYE[df_4_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_4_EYE[df_4_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_4_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_4_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilLeft'] == -1].shape[0] / df_4_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_4_EYE[df_4_EYE['ET_PupilRight'] == -1].shape[0] / df_4_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_4_EYE[df_4_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_4_EYE[df_4_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_4_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_4_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_4_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_4_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_4_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_4_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_4_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_4_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_4_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_4_EYE['Timestamp'], df_4_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_4_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_4_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_4_EYE[col].fillna(df_4_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_4_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_4_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_4_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 5

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_5_EYE = pd.read_csv('data/STData/5/5_EYE.csv')

In [ ]:
df_5_EYE.head()

In [ ]:
df_5_EYE.shape

In [ ]:
df_5_EYE.columns

In [ ]:
df_5_EYE.info()

In [ ]:
df_5_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_5_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_5_EYE['QuestionKey'].unique()

In [ ]:
df_5_EYE['Timestamp'] = pd.to_datetime(df_5_EYE['Timestamp'])

In [ ]:
df_5_EYE.head(3)

In [ ]:
df_5_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_5_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_5_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_5_EYE.isnull().sum()

In [ ]:
df_5_EYE.dropna(inplace=True)

In [ ]:
df_5_EYE.head()

In [ ]:
df_5_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_5_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_5_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_5_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_5_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_5_EYE['ET_ValidityRight'].unique()

In [ ]:
df_5_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_5_EYE['ET_ValidityLeft'].value_counts().index, y=df_5_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_5_EYE['ET_ValidityRight'].value_counts().index, y=df_5_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_5_EYE['ET_ValidityLeft'] = df_5_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_5_EYE['ET_ValidityRight'] = df_5_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_5_EYE.head(3)

In [ ]:
df_5_EYE.describe()

In [ ]:
df_5_EYE[df_5_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_5_EYE[df_5_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_5_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_5_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilLeft'] == -1].shape[0] / df_5_EYE.shape[0]

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilRight'] == -1].shape[0] / df_5_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_5_EYE[df_5_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_5_EYE[df_5_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_5_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_5_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_5_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_5_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_5_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_5_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_5_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_5_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_5_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_5_EYE['Timestamp'], df_5_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_5_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_5_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_5_EYE[col].fillna(df_5_EYE[col].mean(), inplace=True)

In [ ]:
df_5_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_5_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_5_EYE = pd.read_csv('data/STData/5/5_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_5_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_5_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_5_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_5_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_5_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_5_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_5_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_5_EYE['Timestamp'] = pd.to_datetime(df_5_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_5_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_5_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_5_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_5_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_5_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_5_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_5_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_5_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_5_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_5_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_5_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_5_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_5_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_5_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_5_EYE['ET_ValidityLeft'].value_counts().index, y=df_5_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_5_EYE['ET_ValidityRight'].value_counts().index, y=df_5_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_5_EYE['ET_ValidityLeft'] = df_5_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_5_EYE['ET_ValidityRight'] = df_5_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_5_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_5_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_5_EYE[df_5_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_5_EYE[df_5_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_5_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_5_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilLeft'] == -1].shape[0] / df_5_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_5_EYE[df_5_EYE['ET_PupilRight'] == -1].shape[0] / df_5_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_5_EYE[df_5_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_5_EYE[df_5_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_5_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_5_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_5_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_5_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_5_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_5_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_5_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_5_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_5_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_5_EYE['Timestamp'], df_5_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_5_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_5_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_5_EYE[col].fillna(df_5_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_5_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_5_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_5_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 6

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_6_EYE = pd.read_csv('data/STData/6/6_EYE.csv')

In [ ]:
df_6_EYE.head()

In [ ]:
df_6_EYE.shape

In [ ]:
df_6_EYE.columns

In [ ]:
df_6_EYE.info()

In [ ]:
df_6_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_6_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_6_EYE['QuestionKey'].unique()

In [ ]:
df_6_EYE['Timestamp'] = pd.to_datetime(df_6_EYE['Timestamp'])

In [ ]:
df_6_EYE.head(3)

In [ ]:
df_6_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_6_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_6_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_6_EYE.isnull().sum()

In [ ]:
df_6_EYE.dropna(inplace=True)

In [ ]:
df_6_EYE.head()

In [ ]:
df_6_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_6_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_6_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_6_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_6_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_6_EYE['ET_ValidityRight'].unique()

In [ ]:
df_6_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_6_EYE['ET_ValidityLeft'].value_counts().index, y=df_6_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_6_EYE['ET_ValidityRight'].value_counts().index, y=df_6_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_6_EYE['ET_ValidityLeft'] = df_6_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_6_EYE['ET_ValidityRight'] = df_6_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_6_EYE.head(3)

In [ ]:
df_6_EYE.describe()

In [ ]:
df_6_EYE[df_6_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_6_EYE[df_6_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_6_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_6_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilLeft'] == -1].shape[0] / df_6_EYE.shape[0]

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilRight'] == -1].shape[0] / df_6_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_6_EYE[df_6_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_6_EYE[df_6_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_6_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_6_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_6_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_6_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_6_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_6_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_6_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_6_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_6_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_6_EYE['Timestamp'], df_6_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_6_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_6_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_6_EYE[col].fillna(df_6_EYE[col].mean(), inplace=True)

In [ ]:
df_6_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_6_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_6_EYE = pd.read_csv('data/STData/6/6_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_6_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_6_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_6_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_6_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_6_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_6_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_6_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_6_EYE['Timestamp'] = pd.to_datetime(df_6_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_6_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_6_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_6_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_6_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_6_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_6_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_6_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_6_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_6_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_6_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_6_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_6_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_6_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_6_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_6_EYE['ET_ValidityLeft'].value_counts().index, y=df_6_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_6_EYE['ET_ValidityRight'].value_counts().index, y=df_6_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_6_EYE['ET_ValidityLeft'] = df_6_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_6_EYE['ET_ValidityRight'] = df_6_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_6_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_6_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_6_EYE[df_6_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_6_EYE[df_6_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_6_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_6_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilLeft'] == -1].shape[0] / df_6_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_6_EYE[df_6_EYE['ET_PupilRight'] == -1].shape[0] / df_6_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_6_EYE[df_6_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_6_EYE[df_6_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_6_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_6_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_6_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_6_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_6_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_6_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_6_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_6_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_6_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_6_EYE['Timestamp'], df_6_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_6_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_6_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_6_EYE[col].fillna(df_6_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_6_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_6_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_6_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 7

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_7_EYE = pd.read_csv('data/STData/7/7_EYE.csv')

In [ ]:
df_7_EYE.head()

In [ ]:
df_7_EYE.shape

In [ ]:
df_7_EYE.columns

In [ ]:
df_7_EYE.info()

In [ ]:
df_7_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_7_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_7_EYE['QuestionKey'].unique()

In [ ]:
df_7_EYE['Timestamp'] = pd.to_datetime(df_7_EYE['Timestamp'])

In [ ]:
df_7_EYE.head(3)

In [ ]:
df_7_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_7_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_7_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_7_EYE.isnull().sum()

In [ ]:
df_7_EYE.dropna(inplace=True)

In [ ]:
df_7_EYE.head()

In [ ]:
df_7_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_7_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_7_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_7_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_7_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_7_EYE['ET_ValidityRight'].unique()

In [ ]:
df_7_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_7_EYE['ET_ValidityLeft'].value_counts().index, y=df_7_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_7_EYE['ET_ValidityRight'].value_counts().index, y=df_7_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_7_EYE['ET_ValidityLeft'] = df_7_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_7_EYE['ET_ValidityRight'] = df_7_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_7_EYE.head(3)

In [ ]:
df_7_EYE.describe()

In [ ]:
df_7_EYE[df_7_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_7_EYE[df_7_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_7_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_7_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilLeft'] == -1].shape[0] / df_7_EYE.shape[0]

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilRight'] == -1].shape[0] / df_7_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_7_EYE[df_7_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_7_EYE[df_7_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_7_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_7_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_7_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_7_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_7_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_7_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_7_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_7_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_7_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_7_EYE['Timestamp'], df_7_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_7_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_7_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_7_EYE[col].fillna(df_7_EYE[col].mean(), inplace=True)

In [ ]:
df_7_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_7_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_7_EYE = pd.read_csv('data/STData/7/7_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_7_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_7_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_7_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_7_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_7_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_7_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_7_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_7_EYE['Timestamp'] = pd.to_datetime(df_7_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_7_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_7_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_7_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_7_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_7_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_7_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_7_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_7_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_7_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_7_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_7_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_7_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_7_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_7_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_7_EYE['ET_ValidityLeft'].value_counts().index, y=df_7_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_7_EYE['ET_ValidityRight'].value_counts().index, y=df_7_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_7_EYE['ET_ValidityLeft'] = df_7_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_7_EYE['ET_ValidityRight'] = df_7_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_7_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_7_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_7_EYE[df_7_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_7_EYE[df_7_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_7_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_7_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilLeft'] == -1].shape[0] / df_7_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_7_EYE[df_7_EYE['ET_PupilRight'] == -1].shape[0] / df_7_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_7_EYE[df_7_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_7_EYE[df_7_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_7_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_7_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_7_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_7_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_7_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_7_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_7_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_7_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_7_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_7_EYE['Timestamp'], df_7_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_7_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_7_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_7_EYE[col].fillna(df_7_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_7_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_7_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_7_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 8

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_8_EYE = pd.read_csv('data/STData/8/8_EYE.csv')

In [ ]:
df_8_EYE.head()

In [ ]:
df_8_EYE.shape

In [ ]:
df_8_EYE.columns

In [ ]:
df_8_EYE.info()

In [ ]:
df_8_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_8_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_8_EYE['QuestionKey'].unique()

In [ ]:
df_8_EYE['Timestamp'] = pd.to_datetime(df_8_EYE['Timestamp'])

In [ ]:
df_8_EYE.head(3)

In [ ]:
df_8_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_8_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_8_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_8_EYE.isnull().sum()

In [ ]:
df_8_EYE.dropna(inplace=True)

In [ ]:
df_8_EYE.head()

In [ ]:
df_8_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_8_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_8_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_8_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_8_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_8_EYE['ET_ValidityRight'].unique()

In [ ]:
df_8_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_8_EYE['ET_ValidityLeft'].value_counts().index, y=df_8_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_8_EYE['ET_ValidityRight'].value_counts().index, y=df_8_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_8_EYE['ET_ValidityLeft'] = df_8_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_8_EYE['ET_ValidityRight'] = df_8_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_8_EYE.head(3)

In [ ]:
df_8_EYE.describe()

In [ ]:
df_8_EYE[df_8_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_8_EYE[df_8_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_8_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_8_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilLeft'] == -1].shape[0] / df_8_EYE.shape[0]

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilRight'] == -1].shape[0] / df_8_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_8_EYE[df_8_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_8_EYE[df_8_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_8_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_8_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_8_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_8_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_8_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_8_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_8_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_8_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_8_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_8_EYE['Timestamp'], df_8_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_8_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_8_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_8_EYE[col].fillna(df_8_EYE[col].mean(), inplace=True)

In [ ]:
df_8_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_8_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_8_EYE = pd.read_csv('data/STData/8/8_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_8_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_8_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_8_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_8_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_8_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_8_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_8_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_8_EYE['Timestamp'] = pd.to_datetime(df_8_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_8_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_8_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_8_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_8_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_8_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_8_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_8_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_8_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_8_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_8_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_8_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_8_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_8_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_8_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_8_EYE['ET_ValidityLeft'].value_counts().index, y=df_8_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_8_EYE['ET_ValidityRight'].value_counts().index, y=df_8_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_8_EYE['ET_ValidityLeft'] = df_8_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_8_EYE['ET_ValidityRight'] = df_8_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_8_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_8_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_8_EYE[df_8_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_8_EYE[df_8_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_8_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_8_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilLeft'] == -1].shape[0] / df_8_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_8_EYE[df_8_EYE['ET_PupilRight'] == -1].shape[0] / df_8_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_8_EYE[df_8_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_8_EYE[df_8_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_8_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_8_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_8_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_8_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_8_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_8_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_8_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_8_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_8_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_8_EYE['Timestamp'], df_8_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_8_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_8_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_8_EYE[col].fillna(df_8_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_8_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_8_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_8_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 9

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_9_EYE = pd.read_csv('data/STData/9/9_EYE.csv')

In [ ]:
df_9_EYE.head()

In [ ]:
df_9_EYE.shape

In [ ]:
df_9_EYE.columns

In [ ]:
df_9_EYE.info()

In [ ]:
df_9_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_9_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_9_EYE['QuestionKey'].unique()

In [ ]:
df_9_EYE['Timestamp'] = pd.to_datetime(df_9_EYE['Timestamp'])

In [ ]:
df_9_EYE.head(3)

In [ ]:
df_9_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_9_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_9_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_9_EYE.isnull().sum()

In [ ]:
df_9_EYE.dropna(inplace=True)

In [ ]:
df_9_EYE.head()

In [ ]:
df_9_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_9_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_9_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_9_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_9_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_9_EYE['ET_ValidityRight'].unique()

In [ ]:
df_9_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_9_EYE['ET_ValidityLeft'].value_counts().index, y=df_9_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_9_EYE['ET_ValidityRight'].value_counts().index, y=df_9_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_9_EYE['ET_ValidityLeft'] = df_9_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_9_EYE['ET_ValidityRight'] = df_9_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_9_EYE.head(3)

In [ ]:
df_9_EYE.describe()

In [ ]:
df_9_EYE[df_9_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_9_EYE[df_9_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_9_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_9_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilLeft'] == -1].shape[0] / df_9_EYE.shape[0]

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilRight'] == -1].shape[0] / df_9_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_9_EYE[df_9_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_9_EYE[df_9_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_9_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_9_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_9_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_9_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_9_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_9_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_9_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_9_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_9_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_9_EYE['Timestamp'], df_9_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_9_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_9_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_9_EYE[col].fillna(df_9_EYE[col].mean(), inplace=True)

In [ ]:
df_9_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_9_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_9_EYE = pd.read_csv('data/STData/9/9_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_9_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_9_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_9_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_9_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_9_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_9_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_9_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_9_EYE['Timestamp'] = pd.to_datetime(df_9_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_9_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_9_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_9_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_9_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_9_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_9_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_9_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_9_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_9_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_9_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_9_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_9_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_9_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_9_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_9_EYE['ET_ValidityLeft'].value_counts().index, y=df_9_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_9_EYE['ET_ValidityRight'].value_counts().index, y=df_9_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_9_EYE['ET_ValidityLeft'] = df_9_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_9_EYE['ET_ValidityRight'] = df_9_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_9_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_9_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_9_EYE[df_9_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_9_EYE[df_9_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_9_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_9_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilLeft'] == -1].shape[0] / df_9_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_9_EYE[df_9_EYE['ET_PupilRight'] == -1].shape[0] / df_9_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_9_EYE[df_9_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_9_EYE[df_9_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_9_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_9_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_9_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_9_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_9_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_9_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_9_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_9_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_9_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_9_EYE['Timestamp'], df_9_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_9_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_9_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_9_EYE[col].fillna(df_9_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_9_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_9_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_9_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 10

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_10_EYE = pd.read_csv('data/STData/10/10_EYE.csv')

In [ ]:
df_10_EYE.head()

In [ ]:
df_10_EYE.shape

In [ ]:
df_10_EYE.columns

In [ ]:
df_10_EYE.info()

In [ ]:
df_10_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_10_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_10_EYE['QuestionKey'].unique()

In [ ]:
df_10_EYE['Timestamp'] = pd.to_datetime(df_10_EYE['Timestamp'])

In [ ]:
df_10_EYE.head(3)

In [ ]:
df_10_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_10_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_10_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_10_EYE.isnull().sum()

In [ ]:
df_10_EYE.dropna(inplace=True)

In [ ]:
df_10_EYE.head()

In [ ]:
df_10_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_10_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_10_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_10_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_10_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_10_EYE['ET_ValidityRight'].unique()

In [ ]:
df_10_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_10_EYE['ET_ValidityLeft'].value_counts().index, y=df_10_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_10_EYE['ET_ValidityRight'].value_counts().index, y=df_10_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_10_EYE['ET_ValidityLeft'] = df_10_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_10_EYE['ET_ValidityRight'] = df_10_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_10_EYE.head(3)

In [ ]:
df_10_EYE.describe()

In [ ]:
df_10_EYE[df_10_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_10_EYE[df_10_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_10_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_10_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilLeft'] == -1].shape[0] / df_10_EYE.shape[0]

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilRight'] == -1].shape[0] / df_10_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_10_EYE[df_10_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_10_EYE[df_10_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_10_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_10_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_10_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_10_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_10_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_10_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_10_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_10_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_10_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_10_EYE['Timestamp'], df_10_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_10_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_10_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_10_EYE[col].fillna(df_10_EYE[col].mean(), inplace=True)

In [ ]:
df_10_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_10_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_10_EYE = pd.read_csv('data/STData/10/10_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_10_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_10_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_10_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_10_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_10_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_10_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_10_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_10_EYE['Timestamp'] = pd.to_datetime(df_10_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_10_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_10_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_10_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_10_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_10_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_10_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_10_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_10_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_10_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_10_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_10_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_10_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_10_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_10_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_10_EYE['ET_ValidityLeft'].value_counts().index, y=df_10_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_10_EYE['ET_ValidityRight'].value_counts().index, y=df_10_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_10_EYE['ET_ValidityLeft'] = df_10_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_10_EYE['ET_ValidityRight'] = df_10_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_10_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_10_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_10_EYE[df_10_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_10_EYE[df_10_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_10_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_10_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilLeft'] == -1].shape[0] / df_10_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_10_EYE[df_10_EYE['ET_PupilRight'] == -1].shape[0] / df_10_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_10_EYE[df_10_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_10_EYE[df_10_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_10_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_10_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_10_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_10_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_10_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_10_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_10_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_10_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_10_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_10_EYE['Timestamp'], df_10_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_10_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_10_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_10_EYE[col].fillna(df_10_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_10_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_10_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_10_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()